In [1]:
import PIL
import torch
import glob as gb
import numpy as np
from PIL import Image
import os
import random
import torch.nn as nn
import pandas as pd 

from numpy import dot
from numpy.linalg import norm
import matplotlib.pyplot as plt
import random

from facenet_pytorch import MTCNN, InceptionResnetV1
import torchvision.transforms as transforms

random.seed(1)
np.random.seed(1)

In [2]:
# Hyper parameters

def cos_sim(a,b):
    return 1 - dot(a, b)/(norm(a)*norm(b))

def euc_dis(a,b):
    return norm(a-b)

def l2_error(a,b):
    return sum((a-b)**2)

def load_image(path='', resize=True, size=160):
    img = Image.open(path)
    if resize:
        img = img.resize((size,size), resample=PIL.Image.BICUBIC)
    return img

In [3]:
resnet = InceptionResnetV1(pretrained='vggface2').eval()

In [4]:
def return_embedding(img_path):
    img_as_tensor = transforms.ToTensor()(load_image(img_path))
    img_emb = resnet(img_as_tensor.unsqueeze(0)).squeeze(0).detach().numpy()
    return img_emb

def return_batch_embedding(imgs_path):
    img_array = [transforms.ToTensor()(load_image(path)) for path in imgs_path]
    emb_array = [resnet(img.unsqueeze(0)).squeeze(0).detach().numpy() for img in img_array]
    return np.array(emb_array)

In [45]:
img_path = '/media/angelo/DATEN/Datasets/Dados_TCC_Joao/Material TCC/Classe3/Classe/faces_bd/'

gallery_path = gb.glob(img_path + 'gallery-NoSizeMargin/*.png')
gallery_names = [os.path.splitext(name)[0] for name in os.listdir(img_path + 'gallery-NoSizeMargin/')]

face_embeddings = return_batch_embedding(gallery_path)

In [46]:
# Experimentation

probe_path = img_path + 'probe-NoSizeMargin/'
class_folder = [folder for folder in os.listdir(probe_path) if os.path.isdir(os.path.join(probe_path, folder))] 

for classe in class_folder:
    
    accuracy = 0
    class_probe_path = probe_path + classe + '/'
    student_pool_list = os.listdir(class_probe_path)

    for index, student in enumerate(student_pool_list):

        student_emb = return_embedding(class_probe_path + student)

        similarities = [cos_sim(student_emb, face_embeddings[i]) for i in range(len(face_embeddings))]

        least_dis = np.argmin(np.array(similarities))

        if student[:2] == gallery_names[least_dis][:2]:
            accuracy += 1

        #plt.figure(figsize=(5,5))
        #plt.subplot(1,2,1)
        #plt.imshow(load_image(gallery_path[least_dis]))
        #plt.subplot(1,2,2)
        #plt.imshow(load_image(class_probe_path + student))

    print('Accuracy for Class ' + classe + ' was {:.4f}'.format(accuracy/len(student_pool_list)))

Accuracy for Class Classe_1 was 0.5625
Accuracy for Class Classe_2 was 0.6250
Accuracy for Class Classe_3 was 0.5625
